<a href="https://colab.research.google.com/github/Bibliophiles/Machine_Learning/blob/main/weed_ai_yolov5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Train YOLOv5 (and v8 soon!) on Weed-AI Datasets

This guide will take you through training a state-of-the-art object detection architecture - YOLOv5 - on Weed-AI datasets. It combines elements of the official Ultralytics guide, with elements of other custom training and conversion guides.

**Steps:**
1. Setup the project: creating folders, cloning YOLOv5
2. Download the Weed-AI dataset
3. Convert weedCOCO to YOLO annotation format
4. Create YOLOv5 supporting files
5. Train YOLOv5
6. Inference on pictures/videos

The tutorial requires you to have access to a Google Drive account and be able to upload images/data to specific folders. Algorithms will train fastest with a GPU. Select the GPU type under 'Runtime' > 'Change Runtime Type'. Make sure GPU is selected. Premium or High RAM will improve speed/size of models that can be trained.
Make sure you run each cell in the tutorial by pressing the 'Play' button on the left hand side. Some options that may need changing are in capital letters.



In [22]:
# mount google drive - this gives the Colab notebook access to your Drive. It may ask you for permission/to sign in too.
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Create Project Folder

To begin, create a project folder in your Google Drive. We'll call this one `weedai_yolo`. Replace this with whatever name you decide.

It will be created in the root folder of your Google Drive. InTO this folder we'll be cloning the [YOLOv5 GitHub Repository](https://github.com/ultralytics/yolov5) and saving our data too. There are many guides on training YOLOv5 that are accessible through the official repository, make sure to check those for any tips/tricks on tuning your model.

In [2]:
YOUR_DIRECTORY = 'weedai_yolo'

!mkdir /content/drive/MyDrive/{YOUR_DIRECTORY}
%ls '/content/drive/MyDrive/' # should list everything in your Google Drive - double check that your project folder is there.

 cofachievement.pages
 cofachievement.pdf
'Colab Notebooks'/
'Contact Information (1).gform'
'Contact Information (2).gform'
'Contact Information.gform'
 coverletterwithqrcodeandlinks.pdf
'CS1 TUTORIAL SESSION.gdoc'
'CSS APPOINTEES APPLICATION FORM 24 25 (File responses)'/
'CSS APPOINTEES APPLICATION FORM 24 25.gform'
'CSS APPOINTEES APPLICATION FORM 24 25 (Responses).gsheet'
'CSS APPOINTEES FORM.gdoc'
'Dailymotion Cover Letter.gdoc'
 Dennis_A_Anane_C++Story.gdoc
'DennisAnaneBoAGlobalTechnology (1).gdoc'
 DennisAnaneBoAGlobalTechnology.gdoc
 DennisAnaneBoAGlobalTechnology.pdf
 Dennis_Anane_CoverLetter_APPLE.gdoc
'DennisAnaneCoverLetterGoldmanSachs (1).gdoc'
'DennisAnaneCoverLetterGoldmanSachs (1).pdf'
 DennisAnaneCoverLetterGoldmanSachs.gdoc
 DennisAnaneCoverLetterGoldmanSachs.pdf
'Dennis Anane CoverLetter InstaDeep.gdoc'
 Dennis_Anane_Cover_Letter_Junia.gdoc
'Dennis Anane Cover Letter Meltwater.gdoc'
 Dennis_Anane_CoverLetterMistralAI.gdoc
'Dennis Anane Cover Letter MorganStanley.gdoc

**(first time only)**

Clone the YOLOv5 repository so we can use it to train our models. Only do this ONCE at the start of the project.

In [3]:
%cd /content/drive/MyDrive/{YOUR_DIRECTORY}
!git clone https://github.com/ultralytics/yolov5 # clone the YOLOv5 repository. It is a large repository and may take some time depending on your internet speed.

/content/drive/MyDrive/weedai_yolo
Cloning into 'yolov5'...
remote: Enumerating objects: 17521, done.
remote: Counting objects: 100% (24/24), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 17521 (delta 9), reused 0 (delta 0), pack-reused 17497 (from 4)
Receiving objects: 100% (17521/17521), 16.62 MiB | 13.72 MiB/s, done.
Resolving deltas: 100% (12005/12005), done.


# Downloading a Weed-AI dataset

For this example I've used the [Northern WA Wheatbelt Blue Lupins](https://weed-ai.sydney.edu.au/datasets/9df290f4-a29b-44b2-9de6-24bca1cee846) dataset but any of the other object detection datasets would work too, including the recrntly uploaded [Amsinckia in chickpeas](https://weed-ai.sydney.edu.au/datasets/21675efe-9d25-4096-be76-3a541475efd4) dataset.

Download the dataset to a default place on your computer and unzip it. Rename it to something more memorable, in this case `blue_lupins`. Then, we'll create a folder called `datasets` in the `yolov5` directory and move the Weed-AI download (now called `blue_lupins`) to that folder.

To summarise, the steps we will follow below are:
1. Download the dataset on Weed-AI by clicking the button 'Download in WEEDCOCO format'
2. Unzip the download and rename it to something memorable, in this case I've called it `blue_lupins`
3. Create the `datasets` folder in the `yolov5` directory using the code below
4. Move the Weed-AI download into the Google Drive `yolov5/datasets` folder. For me, this is now `'weedai_yolo/yolov5/datasets'`
5. Convert the data from WeedCOCO to [YOLOv5 format](https://roboflow.com/formats/yolov5-pytorch-txt)

Assuming you've downloaded the dataset, unzipped it and changed its name, I'll go through each of these other steps in more detail below.

In [4]:
YOUR_DATASET = 'blue_lupins' # this should match the memorable name of the Weed-AI download you just created.



Create the dataset folder where you'll move the unzipped folder renamed to `blue_lupins` to.

In [5]:
!mkdir /content/drive/MyDrive/{YOUR_DIRECTORY}/yolov5/datasets

In [6]:
!ls /content/drive/MyDrive/{YOUR_DIRECTORY}/yolov5/datasets

blue_lupins


Once the dataset has downloaded and is in the datasets folder, it should have a similar structure to the following:
* yolov5/datasets
    * blue_lupins
        * images
        * weedcoco.json


# Convert weedCOCO to YOLO

The first step in the process is converting the downloaded weedCOCO dataset into the YOLO .txt format. The method below is adapted from the official [Ultralytics GitHub repository](https://github.com/ultralytics/JSON2YOLO/blob/master/labelbox_json2yolo.py). Don't worry too much about the code, though certainly check it out, just run the cell by pressing 'Play' on the left side.

In [7]:
import os
from pathlib import Path

import yaml
import shutil
from tqdm import tqdm
import contextlib
import json

import pandas as pd
import numpy as np
from PIL import Image
from collections import defaultdict

def make_dirs(dir='new_dir/'):
    # Create folders
    dir = Path(dir)
    for p in dir, dir / 'labels', dir / 'images':
        p.mkdir(parents=True, exist_ok=True)  # make dir
    return dir


def convert_weedcoco_json(json_dir=''):
    save_dir = make_dirs(dir=f'{json_dir}')  # output directory
    print()

    # Import json
    for json_file in sorted(Path(json_dir).resolve().glob('*.json')):
        fn = Path(save_dir) # / 'labels' # folder name
        fn.mkdir(exist_ok=True)
        with open(json_file) as f:
            data = json.load(f)

        # Create image dict
        images = {'%g' % x['id']: x for x in data['images']}
        # Create image-annotations dict
        imgToAnns = defaultdict(list)
        for ann in data['annotations']:
            imgToAnns[ann['image_id']].append(ann)


        # Write labels file
        for img_id, anns in tqdm(imgToAnns.items(), desc=f'Annotations {json_file}'):
            # print(img_id, anns)
            img = images['%g' % img_id]
            h, w, f = img['height'], img['width'], img['file_name']

            bboxes = []
            segments = []
            for ann in anns:
                # The COCO box format is [top left x, top left y, width, height]
                box = np.array(ann['bbox'], dtype=np.float64)
                box[:2] += box[2:] / 2  # xy top-left corner to center
                box[[0, 2]] /= w  # normalize x
                box[[1, 3]] /= h  # normalize y
                if box[2] <= 0 or box[3] <= 0:  # if w <= 0 and h <= 0
                    continue

                cls = ann['category_id']  # class
                box = [cls] + box.tolist()
                if box not in bboxes:
                    bboxes.append(box)

            # Write
            with open((fn / f.replace('images', 'labels')).with_suffix('.txt'), 'a') as file:
                for i in range(len(bboxes)):
                    line = *(bboxes[i]),  # cls, box or segments
                    file.write(('%g ' * len(line)).rstrip() % line + '\n')

    # Save dataset.yaml
    names = [data['categories'][i]['name'].split(': ')[1] for i in range(len(data['categories']))]
    d = {'path': json_dir,
         'train': 'images/train',
         'val': 'images/train',
         'test': 'images/train',
         'nc': len(names),
         'names': names}  # dictionary

    with open(f"{save_dir}/weedcoco.yaml", 'w') as f:
        yaml.dump(d, f, sort_keys=False)


    print('\nweedCOCO to YOLO conversion completed successfully!')


In [8]:
WEED_COCO_LOCATION = f"/content/drive/MyDrive/{YOUR_DIRECTORY}/yolov5/datasets/{YOUR_DATASET}"
#convert the weedcoco file
convert_weedcoco_json(json_dir=WEED_COCO_LOCATION)

Annotations /content/drive/MyDrive/weedai_yolo/yolov5/datasets/blue_lupins/weedcoco.json: 100%|██████████| 217/217 [00:01<00:00, 168.01it/s]


weedCOCO to YOLO conversion completed successfully!


## Splitting the dataset into train/validation/test
An algorithm needs a training portion and a validation portion to check as it learns. The test portion is left entirely unseen and can be used later for more appropriate results and to make sure the algorithm hasn't overfit.

If you find the algorithm performs well on the training data but terribly on the val/test data, then it is likely overfitting. This is more common on small datasets and larger models when trained for many epochs.

In [9]:
from sklearn.model_selection import train_test_split

# Read images and annotations
images = [os.path.join(f'{WEED_COCO_LOCATION}/images', x) for x in os.listdir(f'{WEED_COCO_LOCATION}/images')]
annotations = [os.path.join(f'{WEED_COCO_LOCATION}/labels', x) for x in os.listdir(f'{WEED_COCO_LOCATION}/labels') if x[-3:] == "txt"]

images.sort()
annotations.sort()

# Split the dataset into train-val-test splits 80-10-10%
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

%cd {WEED_COCO_LOCATION}
!mkdir images/train images/val images/test labels/train labels/val labels/test

/content/drive/MyDrive/weedai_yolo/yolov5/datasets/blue_lupins


In [10]:
#Utility function to move images
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

# Move the splits into their folders
move_files_to_folder(train_images, 'images/train')
move_files_to_folder(val_images, 'images/val/')
move_files_to_folder(test_images, 'images/test/')
move_files_to_folder(train_annotations, 'labels/train/')
move_files_to_folder(val_annotations, 'labels/val/')
move_files_to_folder(test_annotations, 'labels/test/')

In [11]:
# Check the images have been moved
print(len(os.listdir('images/train')), len(os.listdir('labels/train')))
print(len(os.listdir('images/val')), len(os.listdir('labels/val')))
print(len(os.listdir('images/test')), len(os.listdir('labels/test')))

173 173
22 22
22 22


# Preparing for training
Now we have all the splits made, we need to import some packages and install other YOLOv5 requirements before we can start training a model.

In [12]:
# import necessary packages
import torch
from IPython.display import Image  # for displaying images
import os
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(0)

print('torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

torch 2.6.0+cu124 _CudaDeviceProperties(name='Tesla T4', major=7, minor=5, total_memory=15095MB, multi_processor_count=40, uuid=fb02b706-759a-6cca-2089-3d920a88f3d0, L2_cache_size=4MB)


In [13]:
%cd /content/drive/MyDrive/{YOUR_DIRECTORY}/yolov5
!pip install -r requirements.txt

/content/drive/MyDrive/weedai_yolo/yolov5
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 129.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 61.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 79.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 62.8 MB/s eta 0:00:00
  A

In [14]:
# Weights & Biases  (optional) - this will let you track and visualise the training process with a WandB account; however, it isn't necessary
%pip install -q wandb
import wandb
wandb.login()

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: dennisamematekpor (dennisamematekpor-junia) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# YOLOv5 Training

Now we get to train a model! Change the name of your run to whatever you like, and try playing around with things like image size, batch size, epochs and YOLOv5 variant. Larger variants and larger images will probably do better, but require more memory. So if you run out of memory, just reduce image size or model variant size (choose M instead of X) and then try again.

Information on selecting batch size: https://twitter.com/rasbt/status/1617544195220312066


In [15]:
# train YOLOv5m
BATCH = 8
EPOCHS = 30
IMAGE_SIZE = 1280 # (should be one of 320, 640, 1280, 1920)
MODEL = 'm' # (should be one of 'n', 's', 'm', 'l', 'x' and must be in lower case)

# this is the name of your run, and how it will be saved
RUN_NAME = f'{YOUR_DATASET}_TRAIN_B{str(BATCH)}_E{str(EPOCHS)}_SZ{str(IMAGE_SIZE)}_M{MODEL}'

# avoid making any changes to the below, or check the Ultralytics docs for other commands
!python train.py --img {IMAGE_SIZE} --cfg yolov5{MODEL}.yaml --batch {BATCH} --epochs {EPOCHS} --data datasets/{YOUR_DATASET}/weedcoco.yaml --weights yolov5{MODEL}.pt --name {RUN_NAME}

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
wandb: WARNING ⚠️ wandb is deprecated and will be removed in a future release. See supported integrations at https://github.com/ultralytics/yolov5#integrations.
2025-08-10 19:56:11.568846: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754855771.857629    6922 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754855771.934594    6922 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS wh

# Detect
This is where you can run the model you've just trained on a sample video or other dataset to see how it goes. The --source flag below accepts videos, folders of images and images. All you need to do is upload these to the YOLOv5 datasets directory and then specify the name/path below.

In [16]:
DETECTION_FILES = 'blue_lupins/images/test' # e.g. 'test_video.mp4' OR test_image_directory OR test_image.jpg
CONFIDENCE_THRESHOLD = 0.50 # this should be between 0 and 1. It changes the cutoff value for a detection. Lower = more sensitive, higher = less sensitive

!python detect.py --source datasets/{DETECTION_FILES} --weights runs/train/{RUN_NAME}/weights/best.pt --name {RUN_NAME} --img {IMAGE_SIZE} --conf-thres 0.50

detect: weights=['runs/train/blue_lupins_TRAIN_B8_E30_SZ1280_Mm/weights/best.pt'], source=datasets/blue_lupins/images/test, data=data/coco128.yaml, imgsz=[1280, 1280], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_format=0, save_csv=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs/detect, name=blue_lupins_TRAIN_B8_E30_SZ1280_Mm, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-423-g567c6646 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
image 1/22 /content/drive/MyDrive/weedai_yolo/yolov5/datasets/blue_lupins/images/test/01190df9563e3951106f.JPG: 960x1280 15 lupinus cosentiniis, 66.0ms
image 2/22 /content/drive/MyDrive/weedai_yolo/yolov5/datasets/blue_lupins/images/te

In [27]:
!pip install --upgrade huggingface_hub

In [28]:
from huggingface_hub import notebook_login
notebook_login()

In [29]:
from huggingface_hub import create_repo
repo_id = "dennisamematekpor/objectdetectionyolov5" # Replace with your username and desired model name
create_repo(repo_id, exist_ok=True)

RepoUrl('https://huggingface.co/dennisamematekpor/objectdetectionyolov5', endpoint='https://huggingface.co', repo_type='model', repo_id='dennisamematekpor/objectdetectionyolov5')

In [31]:
!ls "/content/drive/MyDrive/weedai_yolo/yolov5/runs/train/blue_lupins_TRAIN_B8_E30_SZ1280_Mm/weights" # Ensure YOUR_DIRECTORY and RUN_NAME are defined


best.pt  last.pt


In [32]:
# from huggingface_hub import push_to_hub


# model_path = "/content/drive/MyDrive/weedai_yolo/yolov5/runs/train/blue_lupins_TRAIN_B8_E30_SZ1280_Mm/weights/best.pt" # Ensure YOUR_DIRECTORY and RUN_NAME are defined
# push_to_hub(
#     path_or_fileobj=model_path,
#     path_in_repo="weights/best.pt",
#     repo_id=repo_id,
#     commit_message="Add initial trained YOLOv5 model"
# )

In [33]:
# Create a requirements.txt file
requirements = [
    "gradio",
    "torch",
    "torchvision",
    "numpy",
    "opencv-python",
    "Pillow",
    "PyYAML",
    "tqdm",
    "ultralytics", # YOLOv5 dependency
    "huggingface_hub", # For accessing the model from the Hub
    "seaborn" # Added seaborn
]

with open("requirements.txt", "w") as f:
    for item in requirements:
        f.write(f"{item}\n")

print("requirements.txt created successfully!")

requirements.txt created successfully!


### Create `README.md`

This file will serve as the description for your Hugging Face Space.

In [34]:
readme_content = """
# YOLOv5 Weed Detection

This is a Gradio application for detecting weeds in images using a fine-tuned YOLOv5 model trained on a Weed-AI dataset.

## How to Use

1.  Upload an image using the interface.
2.  The model will process the image and display the annotated image with detected weeds and their bounding boxes.
3.  A text output will summarize the detected objects, including their class and confidence score.

## Model Information

The model used in this application is a YOLOv5 model fine-tuned on the [Northern WA Wheatbelt Blue Lupins](https://weed-ai.sydney.edu.au/datasets/9df290f4-a29b-44b2-9de6-24bca1cee846) dataset from Weed-AI.

## Files

-   `app.py`: The Python script containing the Gradio application code.
-   `requirements.txt`: Lists the Python dependencies.
-   `weights/best.pt`: The trained YOLOv5 model weights (this will be uploaded separately).
"""

with open("README.md", "w") as f:
    f.write(readme_content)

print("README.md created successfully!")

README.md created successfully!


In [39]:
import gradio as gr
import torch
import cv2
import numpy as np
import os
import shutil
from huggingface_hub import hf_hub_download # Import hf_hub_download

# Access variables from the notebook's global scope
try:
    YOUR_DIRECTORY = get_ipython().user_ns.get('YOUR_DIRECTORY', 'weedai_yolo')
    RUN_NAME = get_ipython().user_ns.get('RUN_NAME', 'blue_lupins_TRAIN_B8_E30_SZ1280_Mm')
    repo_id = get_ipython().user_ns.get('repo_id', 'dennisamematekpor/objectdetectionyolov5') # Get repo_id
except:
    YOUR_DIRECTORY = 'weedai_yolo'
    RUN_NAME = 'blue_lupins_TRAIN_B8_E30_SZ1280_Mm'
    repo_id = 'dennisamematekpor/objectdetectionyolov5' # Default repo_id

# Load the YOLOv5 model
# Ensure the path to your best.pt is correct relative to where the script will be run
# MODEL_PATH = f"/content/drive/MyDrive/{YOUR_DIRECTORY}/yolov5/runs/train/{RUN_NAME}/weights/best.pt"
# model = torch.hub.load('ultralytics/yolov5', 'custom', path=MODEL_PATH, force_reload=True, trust_repo=True)

# Load the YOLOv5 model from Hugging Face Hub
model_filename = "best.pt"
model_path = hf_hub_download(repo_id=repo_id, filename=model_filename)
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True, trust_repo=True)


def yolov5_inference(img, conf_threshold=0.50):
    """
    Runs YOLOv5 inference on an image and returns the annotated image and predictions.

    Args:
        img (numpy.ndarray): Input image as a NumPy array.
        conf_threshold (float): Confidence threshold for detections.

    Returns:
        tuple: A tuple containing:
            - annotated_img (numpy.ndarray): The image with bounding boxes and labels.
            - predictions_text (str): A text summary of the detected objects.
    """
    # Save the input image to a temporary file for YOLOv5 inference
    temp_img_path = "temp_input.jpg"
    cv2.imwrite(temp_img_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

    # Perform inference with the specified confidence threshold
    model.conf = conf_threshold
    results = model(temp_img_path)


    # Get the annotated image
    results_img = results.render()[0] # render() returns a list of images

    # Generate text summary of predictions
    predictions_text = ""
    if len(results.pred[0]) > 0:
        predictions_text = f"Detected Objects (Confidence Threshold: {conf_threshold:.2f}):\n"
        for det in results.pred[0]:
            class_id = int(det[5])
            confidence = float(det[4])
            class_name = model.names[class_id]
            # Get bounding box coordinates
            x_min, y_min, x_max, y_max = det[:4].int().tolist()
            predictions_text += f"- {class_name}: {confidence:.2f} at [{x_min}, {y_min}, {x_max}, {y_max}]\n"
    else:
        predictions_text = f"No objects detected with confidence threshold {conf_threshold:.2f}."

    # Clean up temporary file
    os.remove(temp_img_path)

    return results_img, predictions_text

# Create the Gradio interface
iface = gr.Interface(
    fn=yolov5_inference,
    inputs=[
        gr.Image(type="numpy", label="Upload Image"),
        gr.Slider(minimum=0.0, maximum=1.0, value=0.50, step=0.05, label="Confidence Threshold")
    ],
    outputs=[gr.Image(type="numpy", label="Annotated Image"), gr.Textbox(label="Predictions")],
    title="YOLOv5 Weed Detection",
    description="Upload an image to detect weeds using a trained YOLOv5 model hosted on Hugging Face. Adjust the slider to change the detection confidence threshold."
)

# Launch the interface
iface.launch(debug=True)

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-8-10 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://9d5f72a72853a9c50a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):
/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:906: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://9d5f72a72853a9c50a.gradio.live


### Updated `app.py` with Confidence Threshold Slider

This file contains the code for your Gradio application with the added slider.

In [40]:
import gradio as gr
import torch
import cv2
import numpy as np
import os
from huggingface_hub import hf_hub_download

# Replace with your actual repository ID
repo_id = "dennisamematekpor/objectdetectionyolov5"
model_filename = "best.pt"

# Load the YOLOv5 model from Hugging Face Hub
model_path = hf_hub_download(repo_id=repo_id, filename=model_filename)
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True, trust_repo=True)


def yolov5_inference(img, conf_threshold=0.50):
    """
    Runs YOLOv5 inference on an image and returns the annotated image and predictions.

    Args:
        img (numpy.ndarray): Input image as a NumPy array.
        conf_threshold (float): Confidence threshold for detections.

    Returns:
        tuple: A tuple containing:
            - annotated_img (numpy.ndarray): The image with bounding boxes and labels.
            - predictions_text (str): A text summary of the detected objects.
    """
    # Save the input image to a temporary file for YOLOv5 inference
    temp_img_path = "temp_input.jpg"
    cv2.imwrite(temp_img_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

    # Perform inference with the specified confidence threshold
    model.conf = conf_threshold
    results = model(temp_img_path)


    # Get the annotated image
    results_img = results.render()[0] # render() returns a list of images

    # Generate text summary of predictions
    predictions_text = ""
    if len(results.pred[0]) > 0:
        predictions_text = f"Detected Objects (Confidence Threshold: {conf_threshold:.2f}):\n"
        for det in results.pred[0]:
            class_id = int(det[5])
            confidence = float(det[4])
            class_name = model.names[class_id]
            # Get bounding box coordinates
            x_min, y_min, x_max, y_max = det[:4].int().tolist()
            predictions_text += f"- {class_name}: {confidence:.2f} at [{x_min}, {y_min}, {x_max}, {y_max}]\n"
    else:
        predictions_text = f"No objects detected with confidence threshold {conf_threshold:.2f}."


    # Clean up temporary file
    os.remove(temp_img_path)

    return results_img, predictions_text

# Create the Gradio interface
iface = gr.Interface(
    fn=yolov5_inference,
    inputs=[
        gr.Image(type="numpy", label="Upload Image"),
        gr.Slider(minimum=0.0, maximum=1.0, value=0.50, step=0.05, label="Confidence Threshold")
    ],
    outputs=[gr.Image(type="numpy", label="Annotated Image"), gr.Textbox(label="Predictions")],
    title="YOLOv5 Weed Detection",
    description="Upload an image to detect weeds using a trained YOLOv5 model hosted on Hugging Face. Adjust the slider to change the detection confidence threshold."
)

# To run this locally, you would use:
# iface.launch(debug=True)

# For Hugging Face Spaces, the app runs automatically based on the file structure.
# You don't need iface.launch() in the final app.py for Spaces.

# You can test the interface locally by uncommenting the line below
# iface.launch(debug=True)


# Save the app.py file
with open("app.py", "w") as f:
    # Write the entire script content to app.py
    f.write("""
import gradio as gr
import torch
import cv2
import numpy as np
import os
from huggingface_hub import hf_hub_download

# Replace with your actual repository ID
repo_id = "dennisamematekpor/objectdetectionyolov5"
model_filename = "best.pt"

# Load the YOLOv5 model from Hugging Face Hub
model_path = hf_hub_download(repo_id=repo_id, filename=model_filename)
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True, trust_repo=True)


def yolov5_inference(img, conf_threshold=0.50):
    \"\"\"
    Runs YOLOv5 inference on an image and returns the annotated image and predictions.

    Args:
        img (numpy.ndarray): Input image as a NumPy array.
        conf_threshold (float): Confidence threshold for detections.

    Returns:
        tuple: A tuple containing:
            - annotated_img (numpy.ndarray): The image with bounding boxes and labels.
            - predictions_text (str): A text summary of the detected objects.
    \"\"\"
    # Save the input image to a temporary file for YOLOv5 inference
    temp_img_path = "temp_input.jpg"
    cv2.imwrite(temp_img_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))

    # Perform inference with the specified confidence threshold
    model.conf = conf_threshold
    results = model(temp_img_path)

    # Get the annotated image
    results_img = results.render()[0] # render() returns a list of images

    # Generate text summary of predictions
    predictions_text = ""
    if len(results.pred[0]) > 0:
        predictions_text = f"Detected Objects (Confidence Threshold: {conf_threshold:.2f}):\\n"
        for det in results.pred[0]:
            class_id = int(det[5])
            confidence = float(det[4])
            class_name = model.names[class_id]
            # Get bounding box coordinates
            x_min, y_min, x_max, y_max = det[:4].int().tolist()
            predictions_text += f"- {class_name}: {confidence:.2f} at [{x_min}, {y_min}, {x_max}, {y_max}]\\n"
    else:
        predictions_text = f"No objects detected with confidence threshold {conf_threshold:.2f}."


    # Clean up temporary file
    os.remove(temp_img_path)

    return results_img, predictions_text

# Create the Gradio interface
iface = gr.Interface(
    fn=yolov5_inference,
    inputs=[
        gr.Image(type="numpy", label="Upload Image"),
        gr.Slider(minimum=0.0, maximum=1.0, value=0.50, step=0.05, label="Confidence Threshold")
    ],
    outputs=[gr.Image(type="numpy", label="Annotated Image"), gr.Textbox(label="Predictions")],
    title="YOLOv5 Weed Detection - Blue Lupins",
    description="Upload an image to detect weeds using a trained YOLOv5 model hosted on Hugging Face. Adjust the slider to change the detection confidence threshold."
)

# To run this locally, you would use:
# iface.launch(debug=True)

# For Hugging Face Spaces, the app runs automatically based on the file structure.
# You don't need iface.launch() in the final app.py for Spaces.

# You can test the interface locally by uncommenting the line below
# iface.launch(debug=True)
""")

print("app.py created successfully!")

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2025-8-10 Python-3.11.13 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)

Fusing layers... 
YOLOv5m summary: 212 layers, 20852934 parameters, 0 gradients, 47.9 GFLOPs
Adding AutoShape... 


app.py created successfully!
